In [ ]:
import json
path = "/home/test_yanjunchi/wangshaobo/ICL_Synthesis/generate/llama3.1_gsm8k_rand10p_normal_generate_4shot.json"
with open(path,"r") as f:
    data = json.load(f)
for i in data:
    print("----")
    # print(i["input"])
    print(i["output"])

In [4]:
import re, json
def match_json_blocks(text): 
    json_block_pattern = r"```json\n(.*?)\n```" # 非贪婪匹配 
    matches = re.findall(json_block_pattern, text, re.DOTALL) 
    return matches
def match_curly_braces_json(text): 
    curly_braces_pattern = r"\{.*?###+\{.*?\}.*?\}" # 非贪婪匹配 
    matches = re.findall(curly_braces_pattern, text, re.DOTALL) 
    # print(1)
    return matches
path = "/home/test_yanjunchi/wangshaobo/ICL_Synthesis/generate/llama3.1_gsm8k_rand10p_normal_generate_4shot_prompt3.json"
with open(path,"r") as f:
    data = json.load(f)
# data = data[:10]
d = []
a = 0
for index,i in enumerate(data):
    try:
        m = match_curly_braces_json(i["output"])
    except Exception as e:
        m = match_json_blocks(i["output"])
        try: 
            pass
        except Exception as e:
            pass
    try:
        j = json.loads(m[0])
        a+= 1
        d.append(j)
    except Exception as e:
        pass
# print(a)
# print(d)
with open("llama3_prompt3_parse.json","w") as f:
    json.dump(d,f,indent=4)

In [ ]:
s = """
{
  "question": "Two friends, Emma and Olivia, are sharing some money in the ratio 3:2. If Emma received $480, how much money will Olivia have after giving $120 to her sister?",
  "answer": "To find the total parts of the ratio, we add 3 + 2 = 5. Since Emma received 3 parts out of 5, 1 part is equal to $480 / 3 = $160. Olivia received 2 parts, so she received 2 x $160 = $320. After giving $120 to her sister, Olivia will have $320 - $120 = $200. Answer is ###{200}."
}
"""
json.loads(s)

In [ ]:

import re
import json
# 示例字符串

# 正则表达式匹配
def parse(text):
    pattern = re.compile(
        r"{'question': '(.*?)', 'answer': '(.*?)'}"
    )

    # 匹配所有符合条件的字符串
    matches = pattern.findall(text)
    answers = []
    # 输出结果
    for match in matches:
        question, answer = match
        # print("Question:", question)
        # print("Answer:", answer)
        # print()
        answers.append({
            "question": question,
            "answer": answer
        })
    return answers
path = "/home/test_yanjunchi/wangshaobo/ICL_Synthesis/generate/llama3.1_gsm8k_rand10p_normal_generate_4shot.json"
with open(path,"r") as f:
    data = json.load(f)
print(len(data))
answer = []
for index,i in enumerate(data):
    ans = parse(i["output"])
    answer += ans
print(len(answer))
with open("llama3.1_gsm8k_rand10p_normal_generate_4shot_parse.json","w") as f:
    json.dump(answer,f,indent=4)

In [3]:
from transformers import AutoTokenizer

# 请将 model_name 修改为你的模型路径或名称
model_name = "/home/test_yanjunchi/wangshaobo/ICL_Synthesis/model/LLM-Research/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 定义系统信息（system）和用户指令（user）
system_message = "You are an expert at generating math question-and-answer pairs."
user_message = """
Task: Generate similar math question-and-answer pairs based on the provided reference sample. The generated Q&A should meet the following criteria:

1. **Wide Range of Fields:** The questions should cover diverse mathematical fields, such as algebra, geometry, probability, statistics, calculus, and number theory.
2. **Detailed and Rigorous Reasoning:** For each question, provide a clear, step-by-step solving process. Ensure that each step is mathematically rigorous and logically consistent.
3. **Final Answer Format:** The final answer must be presented exactly in the format `###{{<result>}}`, where `<result>` is the correct numerical answer.
4. **Strict JSON Format:** Output the results in JSON format, with each entry containing two keys: "question" and "answer". The "answer" field should include the entire reasoning process, concluding with the final answer in the specified format.

**Reference Sample:**

{data}

Your task: Please generate at least 3 distinct Q&A pairs following the guidelines above. Each generated Q&A pair should:
- Be diverse in terms of mathematical topics.
- Provide a complete, step-by-step explanation for the solution.
- End the reasoning with the final answer strictly formatted as ###{{<result>}}.

Generate the output strictly in valid JSON format like the reference sample.
"""

# 构造聊天消息列表
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# 调用 apply_chat_template 生成最终 prompt 字符串
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(formatted_prompt)


ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating